Quantité d'alliages au magnesium à introduire dans la fonte pour obtenir du graphite Speroidal 

La relation empirique suivante donne la quantité d'alliage au magnésium nécessaire pour obtenir du graphite sphéroïdal :

$$
Q = P \cdot \frac{0.76 (S - 0.01) + K + t \cdot 10^{-3}}{\frac{R \cdot Mg}{100}} \cdot \left(\frac{T}{1450}\right)^2
$$

Variables et paramètres :

- **Q** : Quantité d'alliage au magnésium à utiliser (en kg)
- **P** : Poids de fonte à traiter (en kg)
- **S** : Taux de soufre de la fonte de base (en %)
- **t** : Temps de séjour prévu pour la fonte après traitement (en minutes)
- **T** : Température de la fonte au moment du traitement (en degrés Celsius)
- **R** : Rendement en magnésium de l'opération (en %)
- **Mg** : Taux de magnésium dans l'alliage (en %)
- **K** : Quantité de magnésium résiduel nécessaire pour que le graphite soit sous forme sphéroïdal (en %)


Poids métriques des principaux éléments contenus 

| Si (g/m) | Mg (g/m) | RE (g/m) | Zr (g/m) |
|----------|----------|----------|----------|
| 130 - 147| 40 - 45  | 3,0 – 3,5| 3 – 3,5  |

Poids métrique de fil : 418 g/m



Dès la fin du traitement GS :

Temps début coulé : 3 min 43 s, 3 min 8 s

Temps fin coulé : 4 min 59 s, 4 min 15 s

In [60]:
import os
import pandas as pd
import numpy as np

from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import gc

In [61]:

# Lecture de la première feuille du fichier Excel : Tableau des inputs 
chemin_fichier = os.path.join('.', 'Sphérodisation.xlsm')
df = pd.read_excel(chemin_fichier, engine='calamine')

# Extraction des Variables
Parametres_name = df.columns[2:]
masse_poche, pct_soufre, duree_attente_poche, tmp_poche, rendement_mg, pct_mg_fil = (pd.to_numeric(df.at[0, param], errors='coerce') for param in Parametres_name[:6])
masse_four, pct_mg_four, masse_fil, masse_mg_fil, pct_mg_visee, pct_mg_perdu_min, masse_grappe, nb_moules = (pd.to_numeric(df.at[0, param], errors='coerce') for param in Parametres_name[6:14])


In [62]:
# Calcule la quantité de Mg à utiliser
def calcul_quantite_mg(P,S,t,T,R,Mg,K) :
    """
    Calcule la Quantité d'alliage au magnésium (en Kg) à introduire dans la fonte pour obtenir du graphite spherodial.
    Args:
    P: Poids de fonte à traiter en Kg.
    S: Taux de souffre de la fonte de base en %.
    t: Temps de séjour en minutes prévu pour la fonte après traitement.
    T: Température (degrés Celsius) de la fonte au moment du traitement, mesurée au couple.
    R: Rendement en magnésium de l'opération en %.
    Mg: Taux en magnésium dans l'alliage en %.
    K: Quantité de magnésium résiduel nécessaire pour que le graphite soit sous forme sphéroïdal en %.

    Returns:
    Q: Quantité d'alliage au magnésium à utiliser en Kg.
    """
    Q = P * (0.76 * (S - 0.01) + K + t * 1e-3) * (T / 1450) ** 2 / (R * Mg / 100)

    return Q

In [68]:
def calcul_longueur_fil_necessaire(
    masse_four, pct_soufre,duree_attente_poche,
    tmp_poche,rendement_mg,pct_mg_fil,pct_mg_four,
    masse_poche,pct_mg_visee, masse_mg_fil, pct_mg_perdu_min, 
    masse_grappe, nb_moules):
    
    # Masse de Mg dans le four de maintien
    masse_mg_four = masse_four * pct_mg_four / 100 # en kg

    # Masse totale après mélange
    masse_total = masse_four + masse_poche # en kg

    # Masse de Mg visée dans le four de maintien
    masse_mg_visee = masse_total * pct_mg_visee / 100 # en kg

    # Débit de coulée du four de maintien
    debit_coulée = nb_moules * masse_grappe / 60  # en kg/min

    # Durée d'attente du four de maintien entre deux ajouts de fonte traitée
    duree_attente_four = masse_poche / debit_coulée  # en min

    # Masse de Mg évanouis durant le traitement GS
    masse_mg_perdu_four = masse_mg_four * pct_mg_perdu_min * duree_attente_four  # en kg

    # Masse et % de Mg à ajouter dans la poche de traitement pour obtenir le Mg visee
    masse_mg_poche = (masse_mg_visee - masse_mg_four + masse_mg_perdu_four) /( 1 - pct_mg_perdu_min * duree_attente_poche ) # en kg
    pct_mg_poche  = masse_mg_poche / masse_poche * 100  # en %


    # Masse de Mg à ajouter dans la poche de traitement pour obtenir le Mg visee
    masse_mg_manquant = calcul_quantite_mg(masse_poche,pct_soufre,duree_attente_poche,tmp_poche,rendement_mg,pct_mg_fil,pct_mg_poche)

    # Longueur de fil pour avoir la masse de Mg manquante
    longueur_fil_necessaire = masse_mg_manquant / (masse_mg_fil* 1e-3)   # en m

    longueur_fil_necessaire = masse_mg_poche / (masse_mg_fil* 1e-3)  # en m
        
    print(masse_mg_manquant,masse_mg_poche)

    
    return longueur_fil_necessaire

In [75]:
# Données initiales
masse_four = 3000  # en kg
pct_soufre = 0.00110 # en %
duree_attente_poche = 4.50 # en min
tmp_poche =  1450 # °C
rendement_mg = 100 # en %
pct_mg_fil = 43/418*100 # en %
pct_mg_four = 0.038  # en %
masse_poche = 1250  # en kg
pct_mg_visee = 0.045  # en %
masse_mg_fil = 43  # en kg/m
pct_mg_perdu_min = 0.001  # en %
masse_grappe = 40  # en kg
nb_moules = 160

# Calcul de la longueur de fil nécessaire
longueur_fil = calcul_longueur_fil_necessaire(
    masse_four, pct_soufre,duree_attente_poche,
    tmp_poche,rendement_mg,pct_mg_fil,pct_mg_four,
    masse_poche,pct_mg_visee, masse_mg_fil, pct_mg_perdu_min, 
    masse_grappe, nb_moules)

print("Longueur de fil nécessaire (en m) :", longueur_fil)


7.398714004882438 0.7894117277749876
Longueur de fil nécessaire (en m) : 18.35841227383692


# Code

In [10]:
import os
import pandas as pd
import numpy as np

from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import gc


In [11]:
Poid_metrique_Mg = 43 #(g/m)
Poid_metrique_fil = 418 #(g/m)
Percent_Mg_in_1m  = Poid_metrique_Mg/Poid_metrique_fil*100 # % de Mg dans 1 m 
Q_visee = 0.045 # % de Mg visée 
Longueur_total_fil = 4400 # Longueur fil foure (m) 
Poids_total_fil = Longueur_total_fil*Poid_metrique_fil # Poids fil foure (kg) 
L = Q_visee/Percent_Mg_in_1m*Longueur_total_fil


# Récuperation des paramètres de chaque recette 
def Give_input_values_per_recipe(chemin_fichier) :
    # Lecture de la première feuille du fichier Excel : Tableau des inputs 
    df = pd.read_excel(chemin_fichier, engine='calamine')

    # Supprimer les lignes entièrement vides de df
    df_input = df.dropna(how='all').reset_index(drop=True)

    # Obtenir les valeurs de la première colonne sous forme de liste
    liste_des_recettes = df_input[df_input.columns[0]].tolist()

    # Insérer le nom de la colonne au début de la liste
    liste_des_recettes.insert(0, df_input.columns[0])

    # Obtenir les types de recettes uniques
    types_de_recettes = df_input[df_input.columns[0]].dropna().unique().tolist()

    # Ajouter le nom de la colonne à la liste des types de recettes
    types_de_recettes.append(df_input.columns[0])

    # Initialiser un dictionnaire pour les contraintes par recettes
    Parametres_par_recettes = {}

    # Parcourir la liste des recettes
    for idx, recette in enumerate(liste_des_recettes):
        # Vérifier si la recette est un type de recette
        if recette in types_de_recettes:
            # Ajouter les contraintes de la recette au dictionnaire
            Parametres_par_recettes[recette] = df_input.iloc[idx:idx+1, 1:].reset_index(drop=True)

    return df, Parametres_par_recettes

# Calcule de la longueur du fil fourré et de la quantité de Mg à utiliser
def calculate_wire_length(P,S,t,T,R,Mg,K,Q_visee) :
    """
    Calcule la longueur de fil foure (en m) à introduire dans la fonte pour obtenir du graphite spherodial.
    Args:
    P: Poids de fonte à traiter en Kg.
    S: Taux de souffre de la fonte de base en %.
    t: Temps de séjour en minutes prévu pour la fonte après traitement.
    T: Température (degrés Celsius) de la fonte au moment du traitement, mesurée au couple.
    R: Rendement en magnésium de l'opération en %.
    Mg: Taux en magnésium dans l'alliage en %.
    K: Quantité de magnésium résiduel nécessaire pour que le graphite soit sous forme sphéroïdal en %.
    Q_visee: Pourcentage de magnésium visée en g.

    Returns:
    Q_t: Quantité d'alliage au magnésium à utiliser en Kg.
    L: La longueur du fil fourré à utiliser en m.
    """
    Q_t = P * (0.76 * (S - 0.01) + K + t * 1e-3) * (T / 1450) ** 2 / (R * Mg / 100)

    Poid_metrique_Mg = 43 #(g/m)
    Poid_metrique_fil = 418 #(g/m) # Q_visee égal % de Mg visée 
    Longueur_total_fil = 4400 # Longueur Total du fil foure (m) 

    Percent_Mg_in_1m  = Poid_metrique_Mg/Poid_metrique_fil*100 # % de Mg dans 1 m 

    L = Q_visee/Percent_Mg_in_1m*Longueur_total_fil
    return Q_t,L


def export_result(df, dossier_data):
    """
    """
    # Créer le chemin complet du nouveau fichier Excel
    fichier_resultats = os.path.join(dossier_data, 'Resultats.xlsx')

    workbook = Workbook()
    feuille = workbook.active 

    # Écrire le DataFrame dans la feuille
    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            feuille.cell(row=r_idx, column=c_idx, value=value)

    # Sauvegarder le classeur
    workbook.save(fichier_resultats)
    workbook.close()
    gc.collect()
    return 

def main_fct(chemin_fichier, dossier_courant):

    # Récupération des paramètres de chaque recette 
    df_input, Parametres_par_recettes = Give_input_values_per_recipe(chemin_fichier)
    df_output = df_input.copy()
    Q_L_name = ['Quantité théorique d\'alliage au magnésium à utiliser', 'Longueur théorique du fil fourré à utiliser']
    df_output[''] = np.nan
    df_output[Q_L_name] = np.nan

    for recette, df in Parametres_par_recettes.items():
        Parametres_name = df.columns[1:]

        # Extraction des Variables
        P, S, t, T, R, Mg, K,Q = (pd.to_numeric(df[param].iloc[0], errors='coerce') for param in Parametres_name)

        # Calcul de la longueur du fil fourré et de la quantité de Mg à utiliser
        Q_t, L = calculate_wire_length(P, S, t, T, R, Mg, K,Q)

        if recette != df_output.columns[0]:
            index_recette = df_output.index[df_output[df_output.columns[0]] == recette].tolist()[0]
            df_output[Q_L_name[0]] = df_output[Q_L_name[0]].astype(object)
            df_output.loc[index_recette, Q_L_name[0]] = Q_L_name[0]
            df_output.loc[index_recette + 1, Q_L_name[0]] = Q_t

            df_output[Q_L_name[1]] = df_output[Q_L_name[1]].astype(object)
            df_output.loc[index_recette, Q_L_name[1]] = Q_L_name[1]
            df_output.loc[index_recette + 1, Q_L_name[1]] = L
        else:
            df_output.loc[0, Q_L_name[0]] = Q_t
            df_output[Q_L_name[1]] = df_output[Q_L_name[1]].astype(object)
            df_output.loc[0, Q_L_name[1]] = L

    export_result(df_output, dossier_courant )
    return 


In [12]:

if __name__ == "__main__":
    chemin_fichier = os.path.join('.', 'Sphérodisation.xlsm')
    # On recupere le chemin du dossier data
    dossier_courant = os.path.dirname(chemin_fichier)
    # Solve problème
    main_fct(chemin_fichier, dossier_courant)